# Code Agents

При создании многоагентных систем, у нас есть компромисс между гибкостью и управляемостью системы. Управляемые системы обычно действуют на основе некоторых прописанных переходов и коммуникаций между агентами, в то время как в гибких системах LLM-агенты могут принимать решения и строить последовательности вызова инструкментов на основе своих знаний и опыта.

Такие системы могут быть построены на основе кодогенерации, когда LLM генерирует код на Python для решения задачи, использующий предоставленные инструменты, который затем выполняется в некоторой контролируемой песочнице.

Для демонстрации такого подхода используем фреймворк [smolagents](https://github.com/huggingface/smolagents).

In [ ]:
%pip install smolagents[mcp]

## Простейший пример

Рассмотрим простейший пример, когда мы даём агенту возможность поиска в интернет. Для этого будем использовать доступный "из коробки" инструмент `DuckDuckGoSearchTool`.

Для работы с YandexGPT будем использовать [OpenAI-совместимое API](https://yandex.cloud/ru/docs/foundation-models/concepts/openai-compatibility). Вы можете попробовать модели YandexGPT 5 или Llama.

> **ВАЖНО**: При использовании YandexGPT 5 необходимо передать ключ `flatten_messages_as_text = True`, поскольку составные запросы, включающие текст и изображения, пока не поддерживаются.

Для начала убедимся, что языковая модель работает:

In [14]:
from smolagents import OpenAIServerModel, CodeAgent, DuckDuckGoSearchTool
import os

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True
)

#ygpt_model([
#    {
#        "role" : "user",
#        "content" : "Расскажи анекдот"
#    }])

Теперь создадим агента с инструментом поиска в интернет, и зададим ему какой-нибудь вопрос:

In [15]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=ygpt_model)

agent.run("Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = web_search(query="лучшие вина к стейку с кровью и их цены")                                            
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'results = web_search(query="лучшие вина к стейку с кровью и их цены")' due to: 
DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

[Step 1: Duration 3.57 seconds| Input tokens: 2,320 | Output tokens: 66]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("1. Бароло - от 2 494 ₽\n2. Каберне Совиньон - от 252 ₽\n3. Аргентинский Мальбек и чилийский        
  Карменер не удалось найти цены.")                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1. Бароло - от 2 494 ₽
2. Каберне Совиньон - от 252 ₽
3. Аргентинский Мальбек и чилийский Карменер не удалось найти цены.

[Step 4: Duration 2.75 seconds| Input tokens: 13,395 | Output tokens: 358]

'1. Бароло - от 2 494 ₽\n2. Каберне Совиньон - от 252 ₽\n3. Аргентинский Мальбек и чилийский Карменер не удалось найти цены.'

Несмотря на то, что мы задали вопрос на русском языке, весь поиск в интернет проводился на английском, и был найден результат из англоязычного сегмента. Почему?

Дело в том, что за кажущейся внешней простотой скрывается большая внутренняя работа. Во фреймворк smolagents заложена [идеология ReAct](https://habr.com/ru/articles/882432/), в которой агент сначала рассуждает и строит план решения, затем по шагам выполняет его, смотрит на результаты и адаптируется. За этим стоит достаточно сложный промптинг, который по умолчанию во фреймворке smolagents сделан на английском языке.

Загрузим русскоязычный пакет промптов:

In [16]:
import yaml
russian_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
agent.prompt_templates = russian_templates
agent.run("Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = web_search("лучшие вина к стейку с кровью и их цены")                                                  
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Вино к стейку — какое вино лучше подойдет к стейку из говядины, какой 
...](https://luding.ru/gastronomy/sochetaemost/vino-pod-steyk/)
Шатобриан (chateaubriand) — самый постный стейк, которому нужна высокая кислотность бургундского пино нуара или 
кьянти. Можно выбрать бордо или насыщенное розе. Стейк- или бифштекс-тартар (tartar de boeuf) ― практически сырое 
мясо, его готовят с приправами, луком, гренками.

[Какое вино подходит к стейку - 
kamushkiguru.ru](https://kamushkiguru.ru/blog/poleznoe/kakoe-vino-podxodit-k-steiku)
В поиске идеального вина к стейку нужно учитывать несколько основных факторов: тип мяса, способ приготовления, силу
и сложность вкусовых нот, а также личные предпочтения. Варианты подходящего вина к стейку очень разнообразны, и 
каждый может выбрать свою идеальную комбинацию. 1. Мраморный стейк (кобе или вагю)

[Разбираемся в стейках и подбираем под них достойные 
вина](https://swn.ru/articles/razbiraemsya-v-steykakh-i-podbiraem-pod-nikh-dostoynye-vina)
К бургеру из говядины беспроигрышным вариантом будут новосветские красные вина: шираз, мальбек, зинфандель и т. д. 
К барбекю или асадо логичнее всего мальбек, шираз или пинотаж.

[Вино к стейку — Купить в Москве, цена вина к стейку в магазине Декантер](https://decanter.ru/wine/steak)
Вино к стейку в магазине Декантер. 14366 наименований в наличии и под заказ. Лучшие цены, отзывы, гарантия 
качества. ☎ +7 (495) 662-58-95.

[Какое вино подобрать к стейку из 
говядины](https://strongwine.ru/articles/vino/gastronomiia/poleznoe-rukovodstvo-po-sochetaniiu-vina-i-steika/)
Филе можно приготовить на гриле, в духовке или на сковороде. Выбор вина: Испанский Темпранильо. Попробуйте 
выдержанную Rioja Reserva. Итальянский Монтепульчано д'Абруццо, чтобы оценить высшее качество. Французская Сира. 
Хорошо сочетается с масляными соусами.

[Ответы Mail: Что лучше взять к огромному стейку с кровью, пиво, вино 
...](https://otvet.mail.ru/question/202558506)
Красное вино! не сладкое, а именно терпкое.... Анатолий Каморин Искусственный Интеллект (229137) 7 лет назад

[Идеальное сочетание: как подобрать вино к 
стейку](https://www.marieclaire.ru/food/idealnoe-sochetanie-kak-podobrat-vino-k-steyku/)
Сомелье столичного ресторана CHEF подобрал 7 наименований вин, которые станут удачным сочетанием к сочному стейку 
из мраморной говядины. Фавориты для мяса из Франции - вина из провинции Бургундия: Поммар - в восточной части и 
Кот-Дю-Рон - в западной. Вот примеры вин, которые сделают ужин незабываемым: Bouchard Aine & Fils, Pommard, 2013.

[3 лучших вина к стейку: наслаждайтесь идеальным сочетанием](https://dzen.ru/a/Z5upJvVWjEdC2LYw)
Мы собрали для вас три великолепных красных сухих вина, которые идеально дополнят вкус мяса! 🔹 Old Bridge Красное 
сухое Арени Резервное. Вино обладает насыщенным вкусом и богатым ароматом, где преобладают ноты тёмных ягод и 
лёгкие пряные оттенки.

[Как правильно подобрать вино к красному мясу. Часть 1. 
Говядина](https://valentinawine.com/kakoe-vino-podoidet-k-myasu/)
Вино обладает бархатными танинами и высокой кислотность, что как нельзя лучше подойдёт к стейку из филе или 
запечённой вырезке с кровью.

[Вино и стейки - fortwine.ru](https://fortwine.ru/blog/sovety-somele/vino-i-steyki/)
Какие вина пить со стейками, кроме классического Мальбека из Аргентины? Если быть предельно простыми и 
откровенными, то любые красные вина, которые вам нравятся.

Out: None

[Step 1: Duration 3.23 seconds| Input tokens: 2,283 | Output tokens: 65]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer([                                                                                                   
      {"name": "Испанский Темпранильо", "price": "от 211 руб. за бутылку 0.75 л"},                                 
      {"name": "Итальянский Монтепульчано д'Абруццо", "price": "от 430 руб. за бутылку 0.75 л"},                   
      {"name": "Французская Сира", "price": "от 314 руб. за бутылку 0.7 л"}                                        
  ])                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'name': 'Испанский Темпранильо', 'price': 'от 211 руб. за бутылку 0.75 л'}, {'name': 
"Итальянский Монтепульчано д'Абруццо", 'price': 'от 430 руб. за бутылку 0.75 л'}, {'name': 'Французская Сира', 
'price': 'от 314 руб. за бутылку 0.7 л'}]

[Step 3: Duration 5.57 seconds| Input tokens: 12,341 | Output tokens: 477]

[{'name': 'Испанский Темпранильо', 'price': 'от 211 руб. за бутылку 0.75 л'},
 {'name': "Итальянский Монтепульчано д'Абруццо",
  'price': 'от 430 руб. за бутылку 0.75 л'},
 {'name': 'Французская Сира', 'price': 'от 314 руб. за бутылку 0.7 л'}]

Видим, что результат получился уже на русском языке.

## Model Context Protocol

В примере выше, мы искали цены где-то в интернет. А как быть, если агенту нужно работать с конкретным рестораном?

Для этого можно использовать технологию [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction). Это по сути дела стандартизированный способ сделать "удалённый RAG" и "удалённый вызов инструментов". В этом случае ресторан может предоставлять своё меню через набор MCP-вызовов. Для этого он должен реализовать MCP-сервер.

В нашем примере предположим, что у нас есть ресторан, с некоторым [меню кухни](data/menu/food.md) и [бара](data/menu/drinks.md). Мы реализовали [простой MCP-сервер](mcp-server/mcp-rest.py), который предоставляет два инструмента:
* `get_food_menu` для получения списка блюд в виде markdown-таблицы
* `get_drinks_menu` - для получения списка напитков

Для запуска сервера необходимо установить [FastMCP](https://github.com/jlowin/fastmcp), и далее в командной строке выполнить:
```bash
fastmcp run -p 8000 -h 127.0.0.1 -t sse mcp-rest.py
```

In [17]:
from smolagents import OpenAIServerModel, CodeAgent
import os
import yaml

russian_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True)

Теперь мы можем использовать MCP-инструменты в нашем агенте:

In [18]:
import yaml
from smolagents.mcp_client import MCPClient

with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as tools:
    agent = CodeAgent(tools=tools, model=ygpt_model)
    agent.prompt_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
    agent.run("Какое самое дорогое блюдо в ресторане?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Какое самое дорогое блюдо в ресторане?                                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = get_food_menu()                                                                                           
  print(menu)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
| Название блюда            | Описание                                                                 | Тип  | 
Цена (руб) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|---------------|  
| Стейк "Бык на взводе"           | Сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей. Подаётся
с золотой солью Гималаев. | Стейк    | 2500          |  
| Стейк "Нежность разъярённой коровы" | Филе-миньон, замаринованное в гранатовом соке и шёпоте шеф-повара. Плюс 
картофель "А-ля крылышки дракона". | Стейк    | 1800          |  
| "Стейк для скромного олигарха" | Говяжья щека, томлёная 12 часов в вине из погреба Наполеона (ну почти). С 
хрустящими чипсами из кабачков. | Стейк    | 1200          |  
| "Лосось в мечтах о Норвегии" | Нежнейший филе-кусок, запечённый под корочкой из "загадочных северных трав" 
(укропа). Подаётся с лимонным бризом. | Рыба     | 1900          |  
| "Осьминог, который видел слишком много" | Гриль-кальмар с чернильным соусом и намёком на средиземноморские тайны.
Для смелых гурманов. | Рыба     | 2200          |  
| "Дорадо под шубой"        | Филе дорадо, завёрнутое в хрустящий картофельный панцирь. С соусом "Тайна Атлантиды".
| Рыба     | 1600          |  
| "Картофель 'Я тоже могу быть трюфелем'" | Нежные дольки, запечённые с розмарином и намёком на дороговизну. 
Подаётся с воздухом Пиреней. | Гарнир   | 600           |  
| "Грибы, которые танцевали на луну" | Шампиньоны-гриль с чесночным соусом и намёком на психоделический опыт. | 
Гарнир   | 450           |  
| "Круассаны, но не те"     | Аппетитные мини-морковки, глазированные мёдом с намёком на парижское детство. | 
Гарнир   | 350           |  
| "Шоколадный апокалипсис"  | Десерт из 72% какао, взрывающий вкусовые рецепторы. С мороженым "Облако рая". | 
Десерт   | 800           |  
| "Чизкейк, который видел Бога" | Воздушный, с карамелью из ангельских слез и крошками "райского печенья". | Десерт
| 750           |  
| "Тирамису для взрослых"   | Классика, но с добавлением "секретного ингредиента" (кофе, а что вы подумали?). | 
Десерт   | 700           |  


Out: None

[Step 1: Duration 2.20 seconds| Input tokens: 2,281 | Output tokens: 63]

...

[Step 2: Duration 5.16 seconds| Input tokens: 5,348 | Output tokens: 360]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(most_expensive_dish)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'name': 'Стейк "Бык на взводе"', 'description': 'Сочный рибай с розовым сердцем, томлёный в 
дыме аргентинских страстей. Подаётся с золотой солью Гималаев.', 'type': 'Стейк', 'price': 2500}

[Step 3: Duration 1.03 seconds| Input tokens: 9,100 | Output tokens: 403]

## Подбор меню для ужина

Теперь попробуем решить более сложную задачу - подобрать меню для ужина таким образом, чтобы вино и основное блюдо хорошо сочетались. Для этого будем использовать [табличку соответствий еды и вина](data/food_wine_table.md):

In [19]:
import pandas as pd
import re 

foodmatch = [ x.strip() for x in open("data/food_wine_table.md", encoding="utf8").readlines()[2:-1]]
foodmatch = [ { "food" : x.split('|')[0].strip(), "wine" : x.split('|')[1].strip()} for x in foodmatch]
df = pd.DataFrame(foodmatch)
df

,food,wine
0,"Баклажаны, запеченые с сыром",Красное вино: «среднетелые»* сухие — Гренаш (Г...
1,Баранина деликатесная (филе или каре ягненка),Красное вино: сухие выдержанные вина из виногр...
2,"Баранина пикантная: жареная, гриль, тушеная — ...",Красные вина: сухие вина из винограда Каберне ...
3,Бефстроганов,"Белые вина: выдержанные в дубе Шардоне, Пино Г..."
4,Блинчики с мясом говядина,"Крепкие напитки: Водка, Полугар, Хреновуха. Т..."
...,...,...
155,"Шашлык из свинины, пикантный — в уксусно-луков...",Красные вина: тихие сухие и полусухие вина из ...
156,Шоколад,"Чай: черный, травяной, ягодно-фруктовый. Не об..."
157,Эклер классический,"Белые вина: более сладкое, чем торт выдержанно..."
158,Яичница — глазунья,Белые вина: сухие легкие вина с невысокой кисл...


Чтобы научить нашего агента искать соответствия еды и вина, нам нужно дать ему соответствующие инструменты. А для этого реализовать какой-то вид поиска поверх нашей таблички.

Будем использовать семантический поиск через YandexGPT Embeddings:

In [20]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(folder_id=os.environ['folder_id'], auth=os.environ['api_key'])
query_model = sdk.models.text_embeddings('query')
doc_model = sdk.models.text_embeddings('doc')

Посчитаем эмбеддинги для всех блюд и вин в табличке:

In [ ]:
df['food_embed'] = df['food'].apply(lambda x : doc_model.run(x))
df['wine_embed'] = df['wine'].apply(lambda x : doc_model.run(x))
df.to_pickle("data/food_wine_table.pkl")

In [21]:
df = pd.read_pickle("data/food_wine_table.pkl")

Теперь собственно опишем функцию поиска подходящей еды к вину `find_matching_food`:

In [22]:
from scipy.spatial.distance import cdist
import numpy as np
from smolagents import tool

@tool
def find_matching_food(wine : str) -> str:
    """
    Найти подходщее блюдо к заданному вину
    Args:
        wine: Описание вина, под которое надо найти блюдо
    """
    wine_embed = query_model.run(wine)
    we = np.array([np.array(x) for x in df['wine_embed']])
    dist = cdist([wine_embed], we, metric='cosine')[0]
    return "Вот наиболее подходящие блюда:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['food']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_food("Сухое белое"))


Вот наиболее подходящие блюда:
1. Нарезка мясная деликатесная варено-копченая (в т.ч. колбаса)
2. Сыры козие молодые
3. Рыба белая речная (судак, щука, жерех, толстолобик, карп)


Аналогичная функция для поиска вина:

In [23]:
@tool
def find_matching_wine(food : str) -> str:
    """
    Найти подходщее вино к заданному блюду
    Args:
        food: Описание блюда, под которое надо найти вино
    """
    food_embed = query_model.run(food)
    fe = np.array([np.array(x) for x in df['food_embed']])
    dist = cdist([food_embed], fe, metric='cosine')[0]
    return "Вот наиболее подходящие вина:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['wine']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_wine('Стейк'))

Вот наиболее подходящие вина:
1. Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.
2. Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).
3. Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Герма

Теперь собираем всё это вместе, и создаём агента, которому доступны как инструменты подбора блюд/вин, так и меню ресторана по протоколу MCP.

In [24]:
with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as res_tools:
    agent = CodeAgent(
        tools=res_tools+[find_matching_food,find_matching_wine], 
        prompt_templates = russian_templates,
        model=ygpt_model)
    agent.prompt_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
    agent.run("""
    Подбери меню для ужина в ресторане из основного блюда, подходящего вина и десерта, и
    посчитай его стоимость.
    Выведи табличку из названий блюд и напитков и их стоимости. Я хочу стейк!
    """)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Подбери меню для ужина в ресторане из основного блюда, подходящего вина и десерта, и                            │
│     посчитай его стоимость.                                                                                     │
│     Выведи табличку из названий блюд и напитков и их стоимости. Я хочу стейк!                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  matching_wine = find_matching_wine(food="стейк")                                                                 
  print(f"Подходящее вино к стейку: {matching_wine}")                                                              
                                                                                                                   
  food_menu = get_food_menu()                                                                                      
  print(f"Меню блюд:\n{food_menu}")                                                                                
                                                                                                                   
  drinks_menu = get_drinks_menu()                                                                                  
  print(f"Меню напитков:\n{drinks_menu}")                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Подходящее вино к стейку: Вот наиболее подходящие вина:
1. Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от
Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые 
аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне 
Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный 
«ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.
2. Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино 
Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж).
3. Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» 
Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Германии), вина из Бордо, вина из Роны 
(Гренаш+Сира+Мурведр), Саперави (как из Грузии, так и из России), Арени (Армения).
Меню блюд:
| Название блюда            | Описание                                                                 | Тип  | 
Цена (руб) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|---------------|  
| Стейк "Бык на взводе"           | Сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей. Подаётся
с золотой солью Гималаев. | Стейк    | 2500          |  
| Стейк "Нежность разъярённой коровы" | Филе-миньон, замаринованное в гранатовом соке и шёпоте шеф-повара. Плюс 
картофель "А-ля крылышки дракона". | Стейк    | 1800          |  
| "Стейк для скромного олигарха" | Говяжья щека, томлёная 12 часов в вине из погреба Наполеона (ну почти). С 
хрустящими чипсами из кабачков. | Стейк    | 1200          |  
| "Лосось в мечтах о Норвегии" | Нежнейший филе-кусок, запечённый под корочкой из "загадочных северных трав" 
(укропа). Подаётся с лимонным бризом. | Рыба     | 1900          |  
| "Осьминог, который видел слишком много" | Гриль-кальмар с чернильным соусом и намёком на средиземноморские тайны.
Для смелых гурманов. | Рыба     | 2200          |  
| "Дорадо под шубой"        | Филе дорадо, завёрнутое в хрустящий картофельный панцирь. С соусом "Тайна Атлантиды".
| Рыба     | 1600          |  
| "Картофель 'Я тоже могу быть трюфелем'" | Нежные дольки, запечённые с розмарином и намёком на дороговизну. 
Подаётся с воздухом Пиреней. | Гарнир   | 600           |  
| "Грибы, которые танцевали на луну" | Шампиньоны-гриль с чесночным соусом и намёком на психоделический опыт. | 
Гарнир   | 450           |  
| "Круассаны, но не те"     | Аппетитные мини-морковки, глазированные мёдом с намёком на парижское детство. | 
Гарнир   | 350           |  
| "Шоколадный апокалипсис"  | Десерт из 72% какао, взрывающий вкусовые рецепторы. С мороженым "Облако рая". | 
Десерт   | 800           |  
| "Чизкейк, который видел Бога" | Воздушный, с карамелью из ангельских слез и крошками "райского печенья". | Десерт
| 750           |  
| "Тирамису для взрослых"   | Классика, но с добавлением "секретного ингредиента" (кофе, а что вы подумали?). | 
Десерт   | 700           |  

Меню напитков:
| Сорт вина / Напиток       | Винодельня / Производитель       | Год | Тип           | Цена за бокал (руб) |  
|------------------------------|--------------------------------------|---------|--------------------|-------------
-----------|  
| Каберне Совиньон         | Château Lafite Rothschild, Франция   | 2018    | Красное вино       | 4500            
|  
| Мерло                    | Marchesi Antinori, Италия            | 2019    | Красное вино       | 2800            
|  
| Пино Нуар                | Domaine de la Romanée-Conti, Франция | 2017    | Красное вино       | 5200            
|  
| Шираз                    | Penfolds, Австралия                  | 2018    | Красное вино       | 3200            
|  
| Мальбек    

[Step 1: Duration 2.91 seconds| Input tokens: 2,418 | Output tokens: 130]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  steak_name = "Стейк \"Бык на взводе\""                                                                           
  steak_price = 2500                                                                                               
                                                                                                                   
  wine_name = "Каберне Совиньон Château Lafite Rothschild"                                                         
  wine_price = 4500                                                                                                
                                                                                                                   
  dessert_name = "Шоколадный апокалипсис"                                                                          
  dessert_price = 800                                                                                              
                                                                                                                   
  total_cost = steak_price + wine_price + dessert_price                                                            
  final_answer(f"Меню ужина:\n{steak_name}: {steak_price} руб.\n{wine_name}: {wine_price} руб.\n{dessert_name}:    
  {dessert_price} руб.\nОбщая стоимость: {total_cost} руб.")                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Меню ужина:
Стейк "Бык на взводе": 2500 руб.
Каберне Совиньон Château Lafite Rothschild: 4500 руб.
Шоколадный апокалипсис: 800 руб.
Общая стоимость: 7800 руб.

[Step 2: Duration 4.60 seconds| Input tokens: 6,758 | Output tokens: 400]

## Заключение

Code Agents - это способ создания достаточно гибких агентов, способных самостоятельно декомпозировать задачу на подзадачи и решать их с помощью написания кода. 